In [22]:
library("lme4")
library("margins")
library("stargazer")
library("emmeans")
library("ggeffects")
library("broom")
library("broom.mixed")
library("MASS")
library("pscl")

In [2]:
options(repr.matrix.max.rows=500, repr.matrix.max.cols=200)

In [3]:
options(dplyr.print_max = 100)

In [4]:
Data_Root <- "/Data/Promotion/revision/"
data_root <- "/Data/Promotion/revision/reg_results_F_M/"

In [23]:
mydata <- read.csv(paste(Data_Root, "reg_data_drop_missing.csv", sep = ''), header = TRUE, stringsAsFactors = TRUE)

In [24]:
nrow(mydata)

[1] 14552304

In [25]:
ncol(mydata)

[1] 85

In [26]:
mydata = mydata[mydata$gender %in% c("Male", "Female"), ]

In [27]:
mydata = mydata[mydata$affiliation_cate != 'unknown', ]

In [28]:
# our analytical sample
nrow(mydata)

[1] 11396752

In [29]:
mydata$pub_year <- as.factor(mydata$pub_year)

In [30]:
mydata <- droplevels(mydata)

In [31]:
summary(mydata$gender)

Female    Male 
4025650 7371102

In [32]:
mydata <- within(mydata, gender <- relevel(gender, ref = 'Male'))
mydata <- within(mydata, authorship_pos <- relevel(authorship_pos, ref = 'last_position'))
mydata <- within(mydata, affiliation_cate <- relevel(affiliation_cate, ref = 'domestic'))
mydata <- within(mydata, self_promotion <- relevel(self_promotion, ref = 'False'))

In [33]:
keywords <- " + Social_Sciences + Materials_Science + Engineering + Chemistry + \
        Biochemistry__Genetics_and_Molecular_Biology + Medicine + Nursing + Agricultural_and_Biological_Sciences + \
        Pharmacology__Toxicology_and_Pharmaceutics + Neuroscience + Business__Management_and_Accounting + \
        Economics__Econometrics_and_Finance + Chemical_Engineering + Physics_and_Astronomy + Computer_Science + \
        Decision_Sciences + Health_Professions + Psychology + Immunology_and_Microbiology + Dentistry + \
        Earth_and_Planetary_Sciences + Environmental_Science + Mathematics + Arts_and_Humanities + Energy + \
        Veterinary + General"

In [34]:
mydata$author_citation_log <- log2(1 + mydata$author_citation)

In [35]:
mydata$total_num_tweets_log <- log2(1 + mydata$total_num_tweets)

In [36]:
mydata$total_retweet_cn_log <- log2(1 + mydata$matched_tid_retweet_cn)

In [37]:
mydata$total_likes_cn_log <- log2(1 + mydata$matched_tid_likes_cn)

In [38]:
mydata$follower_cn_snapshot_log <- log2(1 + mydata$follower_cn_snapshot)
mydata$follower_cn_snapshot_ours_log <- log2(1 + mydata$follower_cn_snapshot_ours)
mydata$follower_cn_snapshot_combine_log <- log2(1 + mydata$follower_cn_snapshot_combine)

In [74]:
mydata$matched_tid_follower_cn_log <- log2(1 + mydata$matched_tid_follower_cn)

## Using the subset of observations where the author is active on Twitter

1. Remove `on_tw` control
2. Still include gender and self_promotion interaction
3. Add follower control

In [75]:
nrow(mydata[mydata$is_active_on_twitter == "True", ])

[1] 618742

In [76]:
mean(mydata[(mydata$gender == "Female") & (mydata$is_active_on_twitter == "True"), 'total_num_tweets'])

[1] 28.25076

In [77]:
mean(mydata[(mydata$gender == "Male") & (mydata$is_active_on_twitter == "True"), 'total_num_tweets'])

[1] 28.98602

#### Fit a model for all years

In [53]:
summary(mydata[mydata$is_active_on_twitter == "True", c('total_num_tweets')])

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
    0.00     2.00     7.00    28.77    21.00 34764.00 

In [46]:
base_str <- "total_num_tweets ~ 1 + gender * self_promotion + authorship_pos + follower_cn_snapshot_log + \
        author_pub_count_cate + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + \
        author_citation_log + pub_year"
equation <- as.formula(paste(base_str, keywords, sep = " "))
m_yes <- glm.nb(equation, data = mydata[mydata$is_active_on_twitter == "True", ])

In [47]:
tidy(m_yes)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.363815394,1.493831e-02,24.3545247,5.191504e-131
genderFemale,0.148483596,5.123532e-03,28.9807099,1.151741e-184
self_promotionTrue,1.235314458,4.404500e-03,280.4664247,0.000000e+00
authorship_posfirst_position,0.109717974,5.723529e-03,19.1696384,6.636843e-82
authorship_posmiddle_position,0.290013813,4.421204e-03,65.5961202,0.000000e+00
authorship_possolo_author,-0.206360353,1.066301e-02,-19.3529261,1.925913e-83
follower_cn_snapshot_log,0.094976732,9.391970e-04,101.1254683,0.000000e+00
author_pub_count_cate,-0.102244042,1.854771e-03,-55.1248905,0.000000e+00
affiliation_rank_cate,-0.008475538,6.624966e-04,-12.7933298,1.786522e-37


In [33]:
# use this search string in Sublime: " \\\\ \n  &" with " &"
stargazer(m_yes, type = "latex", single.row=TRUE, ci = FALSE, report = ('vc*p'), star.cutoffs = c(0.05, 0.01, 0.001))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Sep 05, 2023 - 02:14:42 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & total\_num\_tweets \\ 
\hline \\[-1.8ex] 
 genderFemale & 0.148$^{***}$ \\ 
  & p = 0.000 \\ 
  self\_promotionTrue & 1.235$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posfirst\_position & 0.110$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posmiddle\_position & 0.290$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_possolo\_author & $-$0.206$^{***}$ \\ 
  & p = 0.000 \\ 
  follower\_cn\_snapshot\_log & 0.095$^{***}$ \\ 
  & p = 0.000 \\ 
  author\_pub\_count\_cate & $-$0.102$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_rank\_cate & $-$0.008$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_cateinternational & 0.069$^{***}

In [110]:
# weighted average for factor variable, median for non-factor variables
MEs = ggemmeans(m_yes, terms=c('self_promotion', 'gender', 'pub_year'), typical='median')

In [111]:
MEs

x,predicted,std.error,conf.low,conf.high,group,facet
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>,<fct>
False,4.315860,0.009845945,4.233373,4.399955,Male,2013
False,5.478465,0.007727278,5.396118,5.562069,Male,2014
False,6.433645,0.006592291,6.351053,6.517312,Male,2015
False,7.339296,0.005999963,7.253494,7.426114,Male,2016
False,9.577959,0.005632767,9.472799,9.684285,Male,2017
False,10.873623,0.005405193,10.759036,10.989431,Male,2018
False,5.006716,0.010460461,4.905113,5.110424,Female,2013
False,6.355423,0.008447499,6.251064,6.461525,Female,2014
False,7.463503,0.007373746,7.356414,7.572151,Female,2015


In [112]:
write.csv(MEs, paste(data_root, "gender_return_on_self_promotion_active_sub.csv", sep = ''), row.names=FALSE)

Marginal effects of self-promotion (using atMedian, not AME)

In [107]:
apply(mydata[(mydata$is_active_on_twitter == "True"), c('follower_cn_snapshot_log', 'author_pub_count_cate', 'affiliation_rank_cate', 'num_authors', 'journal_impact', 'author_citation_log')], 2, median)


follower_cn_snapshot_log    author_pub_count_cate    affiliation_rank_cate 
                9.731319                 6.000000                 3.000000 
             num_authors           journal_impact      author_citation_log 
                6.000000                 3.730000                 9.629357

In [108]:
median_data <- mydata[(mydata$is_active_on_twitter == "True"), ]

In [109]:
median_data$follower_cn_snapshot_log <- 9.73131903102506
median_data$author_pub_count_cate <- 6
median_data$affiliation_rank_cate <- 3
median_data$num_authors <- 6
median_data$journal_impact <- 3.73
median_data$author_citation_log <- 9.62935662007961

In [115]:
summary(median_data[, c('authorship_pos', 'affiliation_cate')])

         authorship_pos        affiliation_cate 
 last_position  :145139   domestic     :217870  
 first_position :116814   international:400872  
 middle_position:336573                         
 solo_author    : 20216                         

In [111]:
margins_eff <- margins(m_yes, data = median_data, variables = c("self_promotion"), at = list(gender = c('Male', 'Female'), pub_year = c('2013', '2014', '2015', '2016', '2017', '2018')))


In [112]:
summary(margins_eff)

,factor,gender,pub_year,AME,SE,z,p,lower,upper
,<chr>,<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,self_promotionTrue,Male,2013,11.60269,0.1175633,98.69317,0,11.37227,11.83311
2,self_promotionTrue,Male,2014,14.72822,0.1186471,124.13466,0,14.49568,14.96076
3,self_promotionTrue,Male,2015,17.29611,0.1192600,145.02856,0,17.06237,17.52986
4,self_promotionTrue,Male,2016,19.73085,0.1242635,158.78232,0,19.48730,19.97440
5,self_promotionTrue,Male,2017,25.74924,0.1499301,171.74164,0,25.45538,26.04309
6,self_promotionTrue,Male,2018,29.23248,0.1584127,184.53366,0,28.92200,29.54296
7,self_promotionTrue,Female,2013,11.19146,0.1328909,84.21537,0,10.93100,11.45192
8,self_promotionTrue,Female,2014,14.20621,0.1440089,98.64808,0,13.92395,14.48846
9,self_promotionTrue,Female,2015,16.68309,0.1536724,108.56263,0,16.38189,16.98428


In [114]:
write.csv(summary(margins_eff), paste(data_root, "delta_gender_return_on_self_promotion_active_sub.csv", sep = ''), row.names=FALSE)


#### Fit a model for each year

In [38]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2018'), ])

[1] 209690

In [21]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2017'), ])

[1] 148751

In [22]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2016'), ])

[1] 110544

In [39]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2015'), ])

[1] 77640

In [23]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2014'), ])

[1] 46947

In [40]:
nrow(mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == '2013'), ])

[1] 25170

In [78]:
for (year in c('2013', '2014', '2015', '2016', '2017', '2018')){
    base_str <- "total_num_tweets ~ 1 + gender * self_promotion + authorship_pos + follower_cn_snapshot_log + \
        author_pub_count_cate + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + \
        author_citation_log"
    equation <- as.formula(paste(base_str, keywords, sep = " "))
    m_yes <- glm.nb(equation, data = mydata[(mydata$is_active_on_twitter == "True") & (mydata$pub_year == year), ])
    print(tidy(m_yes))
    MEs = ggemmeans(m_yes, terms=c('self_promotion', 'gender'), typical='median')
    write.csv(MEs, paste(data_root, "gender_return_on_self_promotion_", year, "_active_sub.csv", sep = ''), row.names=FALSE)
}

# A tibble: 41 x 5
   term                                   estimate std.error statistic   p.value
   <chr>                                     <dbl>     <dbl>     <dbl>     <dbl>
 1 (Intercept)                             0.0163   0.0669       0.243 8.08e-  1
 2 genderFemale                            0.272    0.0284       9.59  8.96e- 22
 3 self_promotionTrue                      1.44     0.0234      61.9   0        
 4 authorship_posfirst_position            0.0440   0.0300       1.47  1.42e-  1
 5 authorship_posmiddle_position           0.0873   0.0239       3.65  2.60e-  4
 6 authorship_possolo_author              -0.246    0.0505      -4.88  1.07e-  6
 7 follower_cn_snapshot_log                0.117    0.00506     23.2   1.16e-118
 8 author_pub_count_cate                  -0.0453   0.0100      -4.52  6.08e-  6
 9 affiliation_rank_cate                  -0.00713  0.00360     -1.98  4.75e-  2
10 affiliation_cateinternational           0.0237   0.0214       1.11  2.69e-  1
11 num_au

#### DV = no. retweets / likes (self-promo obs)

In [54]:
nrow(mydata[mydata$self_promotion == "True", ])

[1] 480622

In [72]:
# Ideally, self_promo == True should be a subset of active obs. 
# But the two variables are coded differently - the latter is based on their matching.
nrow(mydata[(mydata$self_promotion == "True") & (mydata$is_active_on_twitter == "True"), ])

[1] 263945

In [71]:
summary(mydata[mydata$self_promotion == "True", c('follower_cn_snapshot_log')])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   -Inf    -Inf   7.358    -Inf  10.177  20.927 

In [69]:
summary(mydata[mydata$self_promotion == "True", c('matched_tid_retweet_cn')])

    Min.  1st Qu.   Median     Mean  3rd Qu.     Max. 
   0.000    0.000    2.000    6.968    6.000 9892.000 

In [75]:
summary(mydata[mydata$self_promotion == "True", c('matched_tid_follower_cn_log')])

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  0.000   8.344   9.767   9.676  11.102  21.533 

In [76]:
base_str <- "matched_tid_retweet_cn ~ 1 + gender + authorship_pos + matched_tid_follower_cn_log + \
        author_pub_count_cate + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + \
        author_citation_log + pub_year"
equation <- as.formula(paste(base_str, keywords, sep = " "))
m_yes <- glm.nb(equation, data = mydata[mydata$self_promotion == "True", ])

In [77]:
tidy(m_yes)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.0813030013,1.744444e-02,-4.6606840,3.151603e-06
genderFemale,0.0860748021,5.166126e-03,16.6613831,2.501716e-62
authorship_posfirst_position,-0.0438423046,6.996887e-03,-6.2659726,3.705056e-10
authorship_posmiddle_position,0.2626570015,6.020518e-03,43.6269764,0.000000e+00
authorship_possolo_author,-0.0838597125,1.261229e-02,-6.6490475,2.949958e-11
matched_tid_follower_cn_log,0.0647284856,1.078771e-03,60.0020650,0.000000e+00
author_pub_count_cate,-0.0328771556,2.258674e-03,-14.5559535,5.353429e-48
affiliation_rank_cate,-0.0071030045,8.420734e-04,-8.4351368,3.308156e-17
affiliation_cateinternational,-0.0006082174,5.009095e-03,-0.1214226,9.033563e-01


In [78]:
base_str <- "matched_tid_likes_cn ~ 1 + gender + authorship_pos + matched_tid_follower_cn_log + \
        author_pub_count_cate + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + \
        author_citation_log + pub_year"
equation <- as.formula(paste(base_str, keywords, sep = " "))
m_yes <- glm.nb(equation, data = mydata[mydata$self_promotion == "True", ])

In [79]:
tidy(m_yes)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-3.401968395,2.263772e-02,-150.2787470,0.000000e+00
genderFemale,-0.007769829,6.269427e-03,-1.2393204,2.152268e-01
authorship_posfirst_position,0.233097077,8.337462e-03,27.9577968,5.299814e-172
authorship_posmiddle_position,-0.529048931,7.271317e-03,-72.7583355,0.000000e+00
authorship_possolo_author,0.103497134,1.490003e-02,6.9461034,3.755136e-12
matched_tid_follower_cn_log,0.330007299,1.379526e-03,239.2179288,0.000000e+00
author_pub_count_cate,-0.092543157,2.752762e-03,-33.6182918,9.066507e-248
affiliation_rank_cate,-0.017660526,1.023400e-03,-17.2567134,9.962217e-67
affiliation_cateinternational,0.085721273,6.084129e-03,14.0893250,4.417641e-45


In [33]:
# use this search string in Sublime: " \\\\ \n  &" with " &"
stargazer(m_yes, type = "latex", single.row=TRUE, ci = FALSE, report = ('vc*p'), star.cutoffs = c(0.05, 0.01, 0.001))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Sep 05, 2023 - 02:14:42 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & total\_num\_tweets \\ 
\hline \\[-1.8ex] 
 genderFemale & 0.148$^{***}$ \\ 
  & p = 0.000 \\ 
  self\_promotionTrue & 1.235$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posfirst\_position & 0.110$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posmiddle\_position & 0.290$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_possolo\_author & $-$0.206$^{***}$ \\ 
  & p = 0.000 \\ 
  follower\_cn\_snapshot\_log & 0.095$^{***}$ \\ 
  & p = 0.000 \\ 
  author\_pub\_count\_cate & $-$0.102$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_rank\_cate & $-$0.008$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_cateinternational & 0.069$^{***}

## Robustness check 1 (not due to coauthors' self-promotion)

Solo-author papers (active subset)

In [28]:
subdata = mydata[mydata$authorship_pos == "solo_author", ]

In [29]:
nrow(subdata)

[1] 176097

In [41]:
nrow(subdata[subdata$is_active_on_twitter == "True", ])

[1] 20216

In [30]:
base_str <- "total_num_tweets ~ 1 + gender * self_promotion + follower_cn_snapshot_combine_log + \
    author_pub_count_cate + affiliation_rank_cate + affiliation_cate + journal_impact + author_citation_log + pub_year"
equation <- as.formula(paste(base_str, keywords, sep = " "))
m_yes <- glm.nb(equation, data = subdata[subdata$is_active_on_twitter == "True", ])


In [31]:
tidy(m_yes)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),-0.842860669,0.073260625,-11.5049615,1.245450e-30
genderFemale,0.143961977,0.032576421,4.4192079,9.906331e-06
self_promotionTrue,1.450545095,0.023872792,60.7614352,0.000000e+00
follower_cn_snapshot_combine_log,0.154236002,0.004749847,32.4717836,2.668945e-231
author_pub_count_cate,-0.077241269,0.009035480,-8.5486624,1.245232e-17
affiliation_rank_cate,-0.013902025,0.003198486,-4.3464390,1.383655e-05
affiliation_cateinternational,0.004329114,0.019766420,0.2190136,8.266395e-01
journal_impact,0.035265962,0.001275587,27.6468570,3.044504e-168
author_citation_log,0.064260873,0.005172701,12.4230800,1.958727e-35


In [32]:
# use this search string in Sublime: " \\\\ \n  &" with " &"
stargazer(m_yes, type = "latex", single.row=TRUE, ci = FALSE, report = ('vc*p'), star.cutoffs = c(0.05, 0.01, 0.001))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Nov 29, 2022 - 04:35:30 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & total\_num\_tweets \\ 
\hline \\[-1.8ex] 
 genderFemale & 0.144$^{***}$ \\ 
  & p = 0.00001 \\ 
  self\_promotionTrue & 1.451$^{***}$ \\ 
  & p = 0.000 \\ 
  follower\_cn\_snapshot\_combine\_log & 0.154$^{***}$ \\ 
  & p = 0.000 \\ 
  author\_pub\_count\_cate & $-$0.077$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_rank\_cate & $-$0.014$^{***}$ \\ 
  & p = 0.00002 \\ 
  affiliation\_cateinternational & 0.004 \\ 
  & p = 0.827 \\ 
  journal\_impact & 0.035$^{***}$ \\ 
  & p = 0.000 \\ 
  author\_citation\_log & 0.064$^{***}$ \\ 
  & p = 0.000 \\ 
  pub\_year2014 & 0.211$^{***}$ \\ 
  & p = 0.00001 \\ 
  pub\_year

## Robustness check 2 (different definition of self-promotion)

Define self-promotion as those happen in less than a day (fitted to active subset)

In [37]:
base_str <- "total_num_tweets ~ 1 + gender * self_promotion_def + authorship_pos + follower_cn_snapshot_combine_log + \
        author_pub_count_cate + affiliation_rank_cate + affiliation_cate + num_authors + journal_impact + \
        author_citation_log + pub_year"
equation <- as.formula(paste(base_str, keywords, sep = " "))
m_yes <- glm.nb(equation, data = mydata[mydata$is_active_on_twitter == "True", ])


In [38]:
tidy(m_yes)

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.599030622,1.542346e-02,38.8389345,0.000000e+00
genderFemale,0.093046541,4.799430e-03,19.3869971,9.937172e-84
self_promotion_defTrue,0.744199211,4.879306e-03,152.5215365,0.000000e+00
authorship_posfirst_position,0.179590733,5.958296e-03,30.1412905,1.395080e-199
authorship_posmiddle_position,0.171725548,4.583121e-03,37.4691292,2.932282e-307
authorship_possolo_author,-0.099390986,1.108423e-02,-8.9668857,3.050164e-19
follower_cn_snapshot_combine_log,0.134726568,9.485293e-04,142.0373237,0.000000e+00
author_pub_count_cate,-0.109338019,1.930348e-03,-56.6416235,0.000000e+00
affiliation_rank_cate,-0.006754288,6.900955e-04,-9.7874679,1.274475e-22


In [39]:
# use this search string in Sublime: " \\\\ \n  &" with " &"
stargazer(m_yes, type = "latex", single.row=TRUE, ci = FALSE, report = ('vc*p'), star.cutoffs = c(0.05, 0.01, 0.001))


% Table created by stargazer v.5.2.3 by Marek Hlavac, Social Policy Institute. E-mail: marek.hlavac at gmail.com
% Date and time: Tue, Nov 29, 2022 - 04:40:09 PM
\begin{table}[!htbp] \centering 
  \caption{} 
  \label{} 
\begin{tabular}{@{\extracolsep{5pt}}lc} 
\\[-1.8ex]\hline 
\hline \\[-1.8ex] 
 & \multicolumn{1}{c}{\textit{Dependent variable:}} \\ 
\cline{2-2} 
\\[-1.8ex] & total\_num\_tweets \\ 
\hline \\[-1.8ex] 
 genderFemale & 0.093$^{***}$ \\ 
  & p = 0.000 \\ 
  self\_promotion\_defTrue & 0.744$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posfirst\_position & 0.180$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_posmiddle\_position & 0.172$^{***}$ \\ 
  & p = 0.000 \\ 
  authorship\_possolo\_author & $-$0.099$^{***}$ \\ 
  & p = 0.000 \\ 
  follower\_cn\_snapshot\_combine\_log & 0.135$^{***}$ \\ 
  & p = 0.000 \\ 
  author\_pub\_count\_cate & $-$0.109$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_rank\_cate & $-$0.007$^{***}$ \\ 
  & p = 0.000 \\ 
  affiliation\_cateinternational 